In [11]:
import os
import re
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [12]:
def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    """마크다운 문서를 계층적으로 분할합니다."""
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection"),
        ("#####", "subsubsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    chunk_idx = 0
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            chunk_idx += 1
            full_title = "" + current_title
            if "section" in metadata:
                full_title += f" / {metadata['section']}"
            if "subsection" in metadata:
                full_title += f" / {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" / {metadata['subsubsection']}"
            if "subsubsubsection" in metadata:
                full_title += f" / {metadata['subsubsubsection']}"

            content = f"[section_path]: {full_title}\n\n{doc.page_content}"
            doc = Document(page_content=content, metadata={
                **doc.metadata,
                "type": "documentation",
                "source": "dev_center_guide_allmd.md",
                "chunk_idx": chunk_idx
            })

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드 및 분할
# allmd 적용시 검색 품질의 급격한 저하 확(495 chunks)
str_md_file = load_markdown_file("data/dev_center_guide_allmd_touched.md") 
docs_markdown = hierarchical_markdown_split(str_md_file)

print(f"마크다운 문서 분할 완료: {len(docs_markdown)}개 청크")

마크다운 문서 분할 완료: 495개 청크


In [13]:
# docs_markdown에서 'launchPurchaseFlow()' 문자열을 포함하는 Documnet 를 출력

cnt = 0
for doc in docs_markdown:
    if 'PNS' in doc.page_content:
        # print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
        print(f"--- doc_index: {cnt} ---")
        print(f"{doc.page_content}[EOD]")  # 처음 200자만 출력
        # print("-" * 40)  # 구분선 
        cnt += 1
        
print(f"'PNS' 문자열을 포함하는 문서 개수: {cnt}")
        
        

--- doc_index: 0 ---
[section_path]: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App Download](v21/sample)
* [11. V21로 원스토어 인앱결제 업그레이드 하기](v21/upgrade)
* [12. Unity에서 

In [43]:
#### Contextual Retrieval ####
import os
from pathlib import Path
from langchain.docstore.document import Document
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def get_contextual_text(whole_document, chunk_content):

    print("🚀 Contextual Retrieval Quick Start Demo")
    print("=" * 50)
    
    # 1. Initialize the LLM
    print("\n1. Initializing LLM (exaone3.5:latest)...")
    llm = ChatOllama(
        model="exaone3.5:latest",
        # model="solar-pro:latest",
        temperature=0.1
    )
        
    # 2. Set up the contextual prompt
    print("2. Setting up contextual prompt...")
    contextual_prompt = PromptTemplate.from_template(
        """<document> 
{WHOLE_DOCUMENT} 
</document> 
다음은 chunk 처리된 Document입니다. chunk의 내용은 전체 문서에서 일부분을 의미합니다.
<chunk> 
{CHUNK_CONTENT}
</chunk> 
- document의 맥락에서 chunk를 간단 명료하게 요약해주세요.
- 요약문의 최대 토큰은 150 이하여야 합니다. 
- 요약문의 목적은 Document 내에 재삽입하여 retriever를 통합 검색 품질을 높이기 위함입니다.
- 요약문은 한글로 작성해 주세요. 단, chunk내에 코드명, 영문 이니셜 혹은 영어 표현이 문서를 요약하는데 반드시 필요한 내용이라면 영어 그대로 포함될 수 있습니다.
- 요약문을 대표할 수 있는 용어(예, 영문 이니셜, 기능명 등)는 요약문 내에 포함해 주세요.
- 주요 코드값은 한국어로 변환하지 마세요.
- 요약문의 끝부분에 문서를 대표할 수 있는 키워드를 추가해주세요.
"""
    )
    
    chain = contextual_prompt | llm | StrOutputParser()
    
    print(f"Whole document length: {len(whole_document)} characters")
    print(f"Chunk content length: {len(chunk_content)} characters")
    
    # 4. Generate contextual information
    print("\n4. Generating contextual information...")
    try:
        context = chain.invoke({
            "WHOLE_DOCUMENT": whole_document,
            "CHUNK_CONTENT": chunk_content
        })
        
        print("x" * 30)
        print(f"✅ Generated context: {context}")
        print("x" * 30)
        
        # 5. Create enhanced document
        print("\n5. Creating enhanced document...")
        enhanced_content = f"[Abstract]: {context}\n\n[Origin]:{chunk_content}"
        
        enhanced_doc = Document(
            page_content=enhanced_content,
            metadata={
                "original_content": chunk_content,
                "contextual_info": context,
                "source": "demo"
            }
        )
        
        print("✅ Enhanced document created successfully!")
        print(f"Original content length: {len(chunk_content)}")
        print(f"Enhanced content length: {len(enhanced_doc.page_content)}")
        
        print("\n📄 Enhanced Document Preview:")
        print("-" * 40)
        print(enhanced_doc.page_content)
        print("-" * 40)
        
        return True
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

str_md_file = load_markdown_file("data/dev_center_guide_allmd_touched.md") 
target_text = """
[section_path]: 07. PNS(Payment Notification Service) 이용하기 / **PNS 상세** / PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)

* **메세지 이름** : Payment Transaction Notification
* **설명** : 사용자의 개별 결제건의 상태(결제완료, 취소)포함된 message를 개발사의 서버로 전송(원스토어의 서버가 개발사가 사전에 정의한 url을 post(with json body) 형식으로 호출함)
* **URI** : 개발자 센터에서 설정한 Payment Notification URL
* **Method** : POST
* **Request Parameters** : N/A
*   **Request Header** :&#x20;  
| Parameter Name | Data Type | Description      |
| -------------- | --------- | ---------------- |
| Content-Type   | String    | application/json |
* Request Body : JSON 형식  
| Element Name       | Data Type     | Description                                                                   |                                       |
| ------------------ | ------------- | ----------------------------------------------------------------------------- | ------------------------------------- |
| msgVersion         | String        | <p>메시지 버전</p><ul><li>개발(Sandbox) : 3.1.0D</li><li>상용(상용테스트) : 3.1.0</li></ul> |                                       |
| clientId           | String        | 앱의 클라이언트 ID                                                                   |                                       |
| productId          | String        | 인앱상품의 상품 ID                                                                   |                                       |
| messageType        | String        | SINGLE\_PAYMENT\_TRANSACTION 고정                                               |                                       |
| purchaseId         | String        | 구매 ID                                                                         |                                       |
| developerPayload   | String        | 구매건을 식별하기 위해 개발사에서 관리하는 식별자                                                   |                                       |
| purchaseTimeMillis | Long          | 원스토어 결제 시스템에서 결제가 완료된 시간(ms)                                                  |                                       |
| purchaseState      | String        | COMPLETED : 결제완료 / CANCELED : 취소                                              |                                       |
| price              | String        | 결제 금액                                                                         |                                       |
| priceCurrencyCode  | String        | 결제 금액 통화코드(KRW, USD, ...)                                                     |                                       |
| productName        | String        | 구매요청 시 개발사가 customized 인앱상품 제목을 설정한 경우 전달                                     |                                       |
| paymentTypeList    | List          | 결제 정보 목록                                                                      |                                       |
| <p><br></p>        | paymentMethod | String                                                                        | 결제 수단 (상세 내용은 아래 paymentMethod 정의 참고) |
| <p><br></p>        | amount        | String                                                                        | 결제 수단 별 금액                            |
| billingKey         | String        | 확장 기능용 결제 키                                                                   |                                       |
| isTestMdn          | Boolean       | 시험폰 여부(true : 시험폰, false : 시험폰 아님)                                            |                                       |
| purchaseToken      | String        | 구매토큰                                                                          |                                       |
| environment        | String        | <p>결제환경</p><ul><li>개발(샌드박스) : SANDBOX</li><li>상용 :COMMERCIAL</li></ul>        |                                       |
| marketCode         | String        | 마켓 구분코드 ( MKT\_ONE : 원스토어, MKT\_STM : 스톰 )                                    |                                       |
| signature          | String        | 본 메시지에 대한 signature                                                           |                                       |  
\
**Example**  
```
{
"msgVersion" : "3.1.0"
"clientId":"0000000001",
"productId":"0900001234",
"messageType":"SINGLE_PAYMENT_TRANSACTION",
"purchaseId":"SANDBOX3000000004564",
"developerPayload":"OS_000211234",
"purchaseTimeMillis":24431212233,
"purchaseState":"COMPLETED",
"price":"10000",
"priceCurrencyCode":"KRW"
"productName":"GOLD100(+20)"
"paymentTypeList":[
{
"paymentMethod":"DCB",
"amount":"3000"
},
{
"paymentMethod":"ONESTORECASH",
"amount":"7000"
}
],
"billingKey" : "36FED4C6E4AC9E29ADAF356057DB98B5CB92126B1D52E8757701E3A261AF49CCFBFC49F5FEF6E277A7A10E9076B523D839E9D84CE9225498155C5065529E22F5",
"isTestMdn" : true,
"purchaseToken" : "TOKEN...",
"environment" : "SANDBOX",
"marketCode" : "MKT_ONE"
"signature" "SIGNATURE..."
}
```  
"""

print(f"total length: {len(str_md_file)}")
print(f"target text length: {len(target_text)}")

get_contextual_text(str_md_file, target_text)



total length: 383563
target text length: 4854
🚀 Contextual Retrieval Quick Start Demo

1. Initializing LLM (exaone3.5:latest)...
2. Setting up contextual prompt...
Whole document length: 383563 characters
Chunk content length: 4854 characters

4. Generating contextual information...
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
✅ Generated context: **요약:**

**PNS Payment Notification**

원스토어는 개발자 서버로 사용자 결제 상태(완료, 취소)를 포함한 메시지를 POST 방식으로 전송합니다. 메시지는 JSON 형식으로 `Payment Transaction Notification` 이름으로 개발자 센터에서 설정한 URL로 발송됩니다. 주요 요소로는 `msgVersion`, `clientId`, `productId`, `purchaseId`, `developerPayload`, `purchaseTimeMillis`, `purchaseState`, `price`, `paymentTypeList`, `billingKey`, `isTestMdn`, `signature` 등이 포함됩니다. 이 서비스는 결제 완료 또는 취소 알림을 제공하여 개발사가 실시간으로 결제 상태를 확인할 수 있게 합니다.

**키워드:** PNS, Payment Notification, JSON, Purchase State Notification
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

5. Creating enhanced document...
✅ Enhanced document created successfully!
Original content length: 4854
Enhanced content leng

True

In [20]:
# fixed_model_name = "deepseek-coder:6.7b"
# fixed_model_name = "exaone3.5:latest"
# fixed_model_name = "mistral:latest"
# fixed_model_name = "llama3:8b"
fixed_model_name = "bge-m3:latest"    
# fixed_model_name = "solar:latest"

def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=fixed_model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

# 모든 문서 통합
total_docs = docs_markdown #+ docs_kotlin
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")
print(f"- 마크다운 문서: {len(docs_markdown)}개")
#print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 임베딩 생성 및 저장
output_dir = "models/faiss_vs_rag_iap_v10_1_" + fixed_model_name[:3]
os.makedirs(output_dir, exist_ok=True)
# embed_and_save(total_docs, output_dir)

총 495개의 문서를 생성하였습니다.
- 마크다운 문서: 495개


In [21]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever

# fixed_model_name_ret = "bge-m3:latest"
# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 30, "fetch_k": 70, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

bm25 = BM25Retriever.from_documents(
    docs_markdown,
    bm25_params={"k1": 1.5, "b": 0.75}
)

bm25.k = 20

ensembled_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],
    weights=[0.5, 0.5]
)


# res = ensembled_retriever.invoke(
#     "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
# )

res = retriever.invoke(
    "원스토어 인앱결제의 PNS의 개념을 설명해주세요",
    # "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
)

# res = bm25.invoke(
#     # "원스토어 인앱결제의 PNS의 개념을 설명해주세요",
#     "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
# )

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    # print('-' * 40)
    idx += 1

/tmp/ipykernel_4504/2413508350.py:19: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model=fixed_model_name)


검색된 문서 수: 30
--- doc_index: 0 ---
[section_path]: 07. PNS(Payment Notification Service) 이용하기 / **개요**

PNS는 Payment Notification Service의 약자입니다. PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 원스토어의 서버가 개별 사용자의 결제 상태(결제 완료, 결제 취소)를 메시지로 전송하여 결제 트랜젝션의 상태를 손실없이 알려주기 위한 용도의 기능입니다. 즉, 개발사가 지정한 서버에서 원스토어가 정의한 규칙에 맞추어 API를 구현하면 해당 API를 원스토어의 결제 담당 서버에서 호출하는 형태입니다.  
Server to Server, 즉 서버간에 데이터를 전송한다고 할지라도 네트워크 문제로 메세지 전송 실패가 발생하기 때문에 200 OK로 응답을 인지하지 못할 경우 반복하여 메시지가 전송될 수 있습니다. 개발사의 서버는 메시지를 수신후 정의된 응답을 하여야 원스토어는 개발사 서버가 정상적으로 메시지를 전달 받았음을 인지합니다.  
결제 트렌젝션의 유형에 따라 아래와 같은 유형의 메세지가 전송됩니다.  
* 인앱상품 결제 또는 결제취소가 발생하면 원스토어가 개발사 서버로 알림을 전송하는 PNS(Payment Notification Service)&#x20;
* 구독 상태가 변경되면 개발사 서버로 알림을 전송하는 SNS (Subscription Notifacation Service)&#x20;  
{% hint style="warning" %}
Notification은 발송/수신 서버의 상태에 따라 지연 또는 유실될 수 있으므로, notification 수신을 기준으로 상품(서비스)을 제공하는 것은 권장하지 않습니다.&#x20;  
정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.  
원스토어는 검증 및 모니터링

In [44]:
#### Import ContextualRetriever Class ####

# ContextualRetriever 클래스를 같은 폴더의 파일에서 import
import sys
sys.path.append('.')
from code_dev.contextual_retriever import ContextualRetriever, create_contextual_retriever

print("✅ ContextualRetriever 클래스 import 완료!")
print("📁 사용 가능한 기능:")
print("   - ContextualRetriever: 메인 클래스")  
print("   - create_contextual_retriever: 파일에서 직접 로드하는 헬퍼 함수")


✅ ContextualRetriever 클래스 import 완료!
📁 사용 가능한 기능:
   - ContextualRetriever: 메인 클래스
   - create_contextual_retriever: 파일에서 직접 로드하는 헬퍼 함수


In [ ]:
# Cell cleaned up - old class definition removed


클래스 정의 완료!


In [45]:
# Import ContextualRetriever 클래스 사용 예시

# import 확인 (이미 위에서 import되었으므로 주석 처리)
# from contextual_retriever import ContextualRetriever, create_contextual_retriever

print("✅ ContextualRetriever 클래스 import 완료!")

# 1. 방법1: 직접 인스턴스 생성
str_md_file = load_markdown_file("data/dev_center_guide_allmd_touched.md") 
contextual_retriever = ContextualRetriever(
    whole_document=str_md_file,
    model_name="exaone3.5:latest",
    temperature=0.1
)

print("\n" + "="*60)
print("✅ 클래스 초기화 완료 (import된 클래스 사용)!")
print("="*60)

# 2. 방법2: 헬퍼 함수 사용 예시 (주석 처리)
# contextual_retriever_2 = create_contextual_retriever(
#     "data/dev_center_guide_allmd_touched.md",
#     model_name="exaone3.5:latest",
#     temperature=0.1
# )


✅ ContextualRetriever 클래스 import 완료!
🤖 LLM 초기화 중... (exaone3.5:latest)
📝 프롬프트 템플릿 설정 중...
⛓️ 체인 구성 중...
🚀 ContextualRetriever 초기화 완료
📄 전체 문서 길이: 383,563 characters
🤖 모델: exaone3.5:latest
🌡️ Temperature: 0.1

✅ 클래스 초기화 완료 (import된 클래스 사용)!


In [46]:
# 2. 단일 청크 처리 테스트 (get_contextual_text는 이제 chunk_content만 받음)

target_text = """
[section_path]: 07. PNS(Payment Notification Service) 이용하기 / **PNS 상세** / PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)

* **메세지 이름** : Payment Transaction Notification
* **설명** : 사용자의 개별 결제건의 상태(결제완료, 취소)포함된 message를 개발사의 서버로 전송(원스토어의 서버가 개발사가 사전에 정의한 url을 post(with json body) 형식으로 호출함)
* **URI** : 개발자 센터에서 설정한 Payment Notification URL
* **Method** : POST
* **Request Parameters** : N/A
*   **Request Header** :&#x20;  
| Parameter Name | Data Type | Description      |
| -------------- | --------- | ---------------- |
| Content-Type   | String    | application/json |
* Request Body : JSON 형식  
| Element Name       | Data Type     | Description                                                                   |                                       |
| ------------------ | ------------- | ----------------------------------------------------------------------------- | ------------------------------------- |
| msgVersion         | String        | <p>메시지 버전</p><ul><li>개발(Sandbox) : 3.1.0D</li><li>상용(상용테스트) : 3.1.0</li></ul> |                                       |
| clientId           | String        | 앱의 클라이언트 ID                                                                   |                                       |
| productId          | String        | 인앱상품의 상품 ID                                                                   |                                       |
| messageType        | String        | SINGLE\_PAYMENT\_TRANSACTION 고정                                               |                                       |
| purchaseId         | String        | 구매 ID                                                                         |                                       |
| developerPayload   | String        | 구매건을 식별하기 위해 개발사에서 관리하는 식별자                                                   |                                       |
| purchaseTimeMillis | Long          | 원스토어 결제 시스템에서 결제가 완료된 시간(ms)                                                  |                                       |
| purchaseState      | String        | COMPLETED : 결제완료 / CANCELED : 취소                                              |                                       |
| price              | String        | 결제 금액                                                                         |                                       |
| priceCurrencyCode  | String        | 결제 금액 통화코드(KRW, USD, ...)                                                     |                                       |
| productName        | String        | 구매요청 시 개발사가 customized 인앱상품 제목을 설정한 경우 전달                                     |                                       |
| paymentTypeList    | List          | 결제 정보 목록                                                                      |                                       |
| <p><br></p>        | paymentMethod | String                                                                        | 결제 수단 (상세 내용은 아래 paymentMethod 정의 참고) |
| <p><br></p>        | amount        | String                                                                        | 결제 수단 별 금액                            |
| billingKey         | String        | 확장 기능용 결제 키                                                                   |                                       |
| isTestMdn          | Boolean       | 시험폰 여부(true : 시험폰, false : 시험폰 아님)                                            |                                       |
| purchaseToken      | String        | 구매토큰                                                                          |                                       |
| environment        | String        | <p>결제환경</p><ul><li>개발(샌드박스) : SANDBOX</li><li>상용 :COMMERCIAL</li></ul>        |                                       |
| marketCode         | String        | 마켓 구분코드 ( MKT\_ONE : 원스토어, MKT\_STM : 스톰 )                                    |                                       |
| signature          | String        | 본 메시지에 대한 signature                                                           |                                       |  
\
**Example**  
```
{
"msgVersion" : "3.1.0"
"clientId":"0000000001",
"productId":"0900001234",
"messageType":"SINGLE_PAYMENT_TRANSACTION",
"purchaseId":"SANDBOX3000000004564",
"developerPayload":"OS_000211234",
"purchaseTimeMillis":24431212233,
"purchaseState":"COMPLETED",
"price":"10000",
"priceCurrencyCode":"KRW"
"productName":"GOLD100(+20)"
"paymentTypeList":[
{
"paymentMethod":"DCB",
"amount":"3000"
},
{
"paymentMethod":"ONESTORECASH",
"amount":"7000"
}
],
"billingKey" : "36FED4C6E4AC9E29ADAF356057DB98B5CB92126B1D52E8757701E3A261AF49CCFBFC49F5FEF6E277A7A10E9076B523D839E9D84CE9225498155C5065529E22F5",
"isTestMdn" : true,
"purchaseToken" : "TOKEN...",
"environment" : "SANDBOX",
"marketCode" : "MKT_ONE"
"signature" "SIGNATURE..."
}
```  
"""

print(f"📊 테스트 청크 길이: {len(target_text):,} characters")
print("\n🔍 단일 청크 처리 시작...")

# get_contextual_text 메서드 호출 (이제 chunk_content만 전달)
enhanced_doc = contextual_retriever.get_contextual_text(target_text)

if enhanced_doc:
    print(f"\n✅ 처리 성공!")
    print(f"📄 메타데이터: {enhanced_doc.metadata}")
else:
    print(f"\n❌ 처리 실패!")


📊 테스트 청크 길이: 4,854 characters

🔍 단일 청크 처리 시작...

🔍 컨텍스트 정보 생성 시작...
📊 청크 길이: 4,854 characters
✅ 생성된 컨텍스트:
**PNS Payment Notification 요약**

Payment Notification 메시지는 원스토어에서 개발사 서버로 사용자 결제 상태(완료, 취소)를 전송합니다. 메시지는 POST 요청으로 JSON 형식으로 개발자 센터에서 설정한 URL로 발송됩니다. 주요 요소는 `msgVersion`, `clientId`, `productId`, `purchaseId`, `developerPayload`, `purchaseTimeMillis`, `purchaseState`, `price`, `priceCurrencyCode`, `paymentTypeList`, `billingKey`, `isTestMdn`, `purchaseToken`, `environment`, `marketCode`, `signature` 등이 포함됩니다. 이 메시지는 결제 완료 또는 취소 상태를 전달하며, 테스트 환경과 상용 환경에 따라 버전이 다릅니다. 요약 용어: PNS, Payment Notification
✅ 향상된 Document 생성 완료!
📏 원본 길이: 4,854 characters
📏 향상된 길이: 5,379 characters
📈 증가율: 10.8%

✅ 처리 성공!
📄 메타데이터: {'original_content': '\n[section_path]: 07. PNS(Payment Notification Service) 이용하기 / **PNS 상세** / PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)\n\n* **메세지 이름** : Payment Transaction Notification\n* **설명** : 사용자의 개별 결제건의 상태(결제완료, 취소)포함된 message를 개발사의 서버로 전송(원스토어의 서버가 개발사가 사전에 정의한

In [ ]:
# 3. 클래스의 추가 기능 테스트

print("🔍 미리보기 기능 테스트:")
print("-" * 50)

# 미리보기 기능 테스트
short_text = """
[section_path]: 07. PNS(Payment Notification Service) 이용하기 / **개요**

PNS는 Payment Notification Service의 약자입니다. PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 원스토어의 서버가 개별 사용자의 결제 상태(결제 완료, 결제 취소)를 메시지로 전송하여 결제 트랜젝션의 상태를 손실없이 알려주기 위한 용도의 기능입니다.
"""

contextual_retriever.preview_enhancement(short_text, max_chars=300)

print("\n\n🔄 여러 청크 일괄 처리 테스트:")
print("-" * 50)

# 여러 청크 일괄 처리 테스트
test_chunks = [
    "PNS는 Payment Notification Service입니다.",
    "원스토어 인앱결제는 다양한 결제 수단을 지원합니다.",
    "개발사는 PNS 메시지를 받아 결제 상태를 확인할 수 있습니다."
]

enhanced_docs = contextual_retriever.process_multiple_chunks(test_chunks, verbose=False)
print(f"📊 처리 결과: {len(enhanced_docs)}개 문서 생성됨")

print("\n\n✅ 모든 테스트 완료!")
print("🎯 클래스 주요 특징:")
print("   - whole_document는 생성자에서 설정")
print("   - get_contextual_text()는 chunk_content만 매개변수로 받음")
print("   - LLM과 프롬프트는 클래스 초기화 시 설정")
print("   - 다양한 유틸리티 메서드 제공 (미리보기, 일괄처리)")


In [ ]:
# 🧪 Import 테스트 및 기능 확인

print("="*60)
print("🧪 ContextualRetriever 모듈 테스트")
print("="*60)

# 1. 클래스 정보 확인
print(f"📋 클래스 타입: {type(ContextualRetriever)}")
print(f"📋 클래스 위치: {ContextualRetriever.__module__}")
print(f"📋 클래스 문서: {ContextualRetriever.__doc__}")

# 2. 헬퍼 함수 정보 확인  
print(f"\n🔧 헬퍼 함수: {create_contextual_retriever.__name__}")
print(f"🔧 함수 문서: {create_contextual_retriever.__doc__}")

# 3. 인스턴스 정보 확인
if 'contextual_retriever' in locals():
    print(f"\n🎯 현재 인스턴스: {contextual_retriever}")
    print(f"📊 인스턴스 통계: {contextual_retriever.get_stats()}")
    print(f"🏷️ 인스턴스 표현: {repr(contextual_retriever)}")
else:
    print("\n⚠️ 아직 인스턴스가 생성되지 않았습니다.")

print("\n✅ 모든 import 테스트 완료!")
